In [1]:
import torch
import torchvision
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.functional import F
from tqdm import trange
import numpy as np
from tqdm import trange

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)


Mounted at /content/drive


In [3]:
%cd '/content/drive/MyDrive/Colab Notebooks/phase4 public'

/content/drive/.shortcut-targets-by-id/1olR9djsPqRY8dxn01k2qeMnqfOfqWRQE/phase4 public


In [4]:
!ls

'CI - Spring_2023 - phase4.pdf'   exploring_data.py   utils.py
 data				  __pycache__
 dataloader_demo.ipynb		  triplet_loss.py


In [4]:
from utils import get_data_loaders
import torch

def loaders_demo():
    full_dataloaders, _ = get_data_loaders(
        filenames={
            'train': './data/12000_train_mnistmnistmsvhnsynusps.npz',
            'test': './data/12000_test_mnistmnistmsvhnsynusps.npz',
        },
        batch_size= 128
    )
    return full_dataloaders

full_dataloaders = loaders_demo()
print(full_dataloaders.keys())

for phase in ['train', 'test', 'test_missing']:
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(full_dataloaders[phase]):
        print(f'{batch_indx}-th batch')
        print('images shape: ', images.shape)
        print('features shape: ', features.shape)
        if phase == 'test_missing':
            print('in test-missing dataloaders, since the features are not available, features are filled with zeros', torch.sum(features))
        print('domain labels freq: ', torch.unique(domain_labels, return_counts=True))
        print('digit labels freq: ', torch.unique(digit_labels, return_counts=True))
        print()
        break

datafiles to read:  {'train': './data/12000_train_mnistmnistmsvhnsynusps.npz', 'test': './data/12000_test_mnistmnistmsvhnsynusps.npz'}
reading ./data/12000_train_mnistmnistmsvhnsynusps.npz, number of samples: 60000
reading ./data/12000_test_mnistmnistmsvhnsynusps.npz, number of samples: 21600
reading ./data/12000_test_mnistmnistmsvhnsynusps.npz, number of samples: 21600
dict_keys(['train', 'test', 'test_missing', 'train_size', 'test_size', 'test_missing_size'])
0-th batch
images shape:  torch.Size([128, 3, 32, 32])
features shape:  torch.Size([128, 256])
domain labels freq:  (tensor([0, 1, 2, 3, 4]), tensor([22, 25, 20, 35, 26]))
digit labels freq:  (tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([11, 14, 14, 14, 13, 17, 13, 11, 13,  8]))

0-th batch
images shape:  torch.Size([128, 3, 32, 32])
features shape:  torch.Size([128, 256])
domain labels freq:  (tensor([0, 1, 2, 3, 4]), tensor([22, 18, 26, 31, 31]))
digit labels freq:  (tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]), tensor([10, 20, 1

In [5]:
torch.set_printoptions(precision=2)

In [6]:
use_gpu = True
device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
device

device(type='cuda')

In [ ]:
len(full_dataloaders['train'].dataset), len(full_dataloaders['test'].dataset)

(60000, 21600)

In [ ]:
for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(full_dataloaders['train']):
    if batch_indx < 1:
        break
import pandas as pd
pd.DataFrame(features)

,0,1,2,3,4,5,6,7,8,9,...,246,247,248,249,250,251,252,253,254,255
0,6.037965,-1.799968,1.182611,-0.248518,-0.320674,0.251610,0.734077,0.608663,0.212084,0.767771,...,-0.072638,0.316827,0.023135,-0.329847,-0.159941,-0.212282,0.070331,-0.110758,-0.065389,-0.018419
1,-12.564800,0.517854,1.038122,-0.489715,-1.271697,-0.718387,2.967587,3.289755,0.988581,-0.101411,...,-0.454926,-0.120079,-0.101010,-0.171803,-0.297529,0.160597,-0.018645,-0.045893,-0.045439,-0.096207
2,7.445361,-0.557834,-1.135024,0.368366,0.614173,0.158659,-0.522470,-0.589508,0.290434,-0.124693,...,-0.135883,-0.080708,0.098077,-0.099581,0.086981,-0.015576,0.033817,-0.171754,-0.028527,-0.012392
3,0.744837,-4.175048,0.368809,0.091318,1.096337,0.079871,0.053634,-0.415834,-0.559677,0.074155,...,-0.032611,-0.089508,-0.032700,-0.127731,-0.191442,-0.135071,-0.055170,0.002007,-0.098564,0.124951
4,0.228644,-3.329779,-2.187275,3.250448,4.089767,-0.171248,-0.909210,-0.898119,-4.993124,-0.198844,...,0.128845,0.038883,-0.115404,0.339894,-0.298737,-0.154749,-0.035553,0.143919,0.050182,0.179309
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,-14.531523,-6.808526,-1.962304,-2.109931,2.470517,0.892024,1.383679,-2.025483,-0.928582,0.343404,...,0.074560,0.105696,-0.235819,-0.009158,0.068220,0.018031,0.079444,-0.026290,0.098710,-0.030104
124,8.207400,5.156352,-3.013297,0.255962,2.002417,-2.155067,4.849430,0.571528,3.321007,-1.156567,...,0.072982,-0.017911,0.056130,-0.089698,-0.018625,0.021709,-0.126845,0.075862,0.179969,0.049354
125,-11.028695,2.798279,-1.152583,-0.779837,3.369474,-0.146005,-0.842274,-4.763255,6.170467,-2.651726,...,-0.179539,0.107593,-0.226426,0.093342,-0.087881,0.130228,0.142368,0.333962,0.225531,0.132795
126,-10.087826,4.747341,0.836692,-5.564118,-1.667153,-1.559360,-3.061188,-0.031718,2.186175,1.350780,...,0.024626,-0.005803,-0.003717,-0.054994,-0.009849,0.031435,-0.002613,0.011187,-0.032867,0.009332


In [ ]:
batch_indx, len(full_dataloaders['train'])

(0, 469)

### Defining Model

In [7]:
class demo_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # kernel
        self.conv1 = torch.nn.Conv2d(3, 6, 5, stride=1, padding=0)
        self.conv2 = torch.nn.Conv2d(6, 16, 3, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(16, 32, 3, stride=1, padding=0)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=3200 + 256, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)

    # Forward Pass
    def forward(self, inputs, inputs2, debug=False):
        x = F.max_pool2d(F.relu(self.conv1(inputs)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 1)
        x = F.max_pool2d(F.relu(self.conv3(x)), 1)
        x0 = self.flatten(x)

        use_gpu = True
        device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
        merged_input = torch.zeros(128, 3456)
        merged_input = merged_input.to(device)

        for i in range(x0.shape[0]):
            merged_input[i] = torch.cat((x0[i], inputs2[i]))


        x1 = self.fc1(merged_input)
        x1 = F.relu(x1)
        x2 = self.fc2(x1)
        x2 = F.relu(x2)
        outputs = self.fc3(x2)

        if debug:
            print('inputs shape: ', inputs.shape) # inputs in shape [N, C, H, W]
            print('after flattening: ', x0.shape)
            print('Activations after 1st fully connected layer: ', x1.shape)
            print('Activations after 2nd fully connected layer: ', x2.shape)
            print('Output shape: ', outputs.shape)

        return outputs

model = demo_model()
print(model)

demo_model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3456, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [ ]:
def train_one_epoch(model: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.train() #
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
        if digit_labels.shape[0] != 128:
            continue
        images = images.to(device)
        features = features.to(device)
        digit_labels = digit_labels.to(device)

        outputs = model(images, features) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, digit_labels) # Compute Loss

        loss.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        running_corrects += torch.sum(preds == digit_labels)
        running_loss += loss.item()

    epoch_acc = (running_corrects / num_samples) * 100
    epoch_loss = (running_loss / num_batches)


    return epoch_acc, epoch_loss

## Evaluating Model:

In [27]:
def test_model(model: nn.Module,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.eval() # you must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference.
    with torch.no_grad():
        for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
            if digit_labels.shape[0] != 128:
                continue
            images = images.to(device)
            features = features.to(device)
            digit_labels = digit_labels.to(device)

            outputs = model(images, features) # Forward Pass
            loss = loss_fn(outputs, digit_labels) # Compute Loss

            _, preds = torch.max(outputs, 1) #
            running_corrects += torch.sum(preds == digit_labels)
            running_loss += loss.item()

    test_acc = (running_corrects / num_samples) * 100
    test_loss = (running_loss / num_batches)

    return test_acc, test_loss

In [ ]:
import matplotlib.pyplot as plt


def custom_plot_training_stats(acc_hist, loss_hist, phase_list, title: str, dir: str):
    fig, (ax1, ax2) = plt.subplots(nrows = 1, ncols = 2, figsize=[14, 6], dpi=100)

    for phase in phase_list:
        lowest_loss_x = np.argmin(np.array(loss_hist[phase]))
        lowest_loss_y = loss_hist[phase][lowest_loss_x]

        ax1.annotate("{:.4f}".format(lowest_loss_y), [lowest_loss_x, lowest_loss_y])
        ax1.plot(loss_hist[phase], '-x', label=f'{phase} loss', markevery = [lowest_loss_x])

        ax1.set_xlabel(xlabel='epochs')
        ax1.set_ylabel(ylabel='loss')

        ax1.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax1.legend()
        ax1.label_outer()

    # acc:
    for phase in phase_list:
        highest_acc_x = np.argmax(np.array(acc_hist[phase].cpu()))
        highest_acc_y = acc_hist[phase][highest_acc_x]

        ax2.annotate("{:.4f}".format(highest_acc_y), [highest_acc_x, highest_acc_y])
        ax2.plot(acc_hist[phase], '-x', label=f'{phase} loss', markevery = [highest_acc_x])

        ax2.set_xlabel(xlabel='epochs')
        ax2.set_ylabel(ylabel='acc')

        ax2.grid(color = 'green', linestyle = '--', linewidth = 0.5, alpha=0.75)
        ax2.legend()
        #ax2.label_outer()

    fig.suptitle(f'{title}')

    plt.show()

In [ ]:
def demo():
    batch_size = 128
    num_epochs = 25
    learning_rate = 0.005

    model = demo_model()
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()

    acc_history = {'train': [], 'test': []}
    loss_history = {'train': [], 'test': []}

    for epoch in trange(num_epochs):
        train_acc, train_loss = train_one_epoch(model=model, optim=optimizer, dataloader=full_dataloaders['train'], loss_fn=cross_entropy)
        test_acc, test_loss = test_model(model=model, dataloader=full_dataloaders['test'], loss_fn=cross_entropy)

        acc_history['train'].append(train_acc)
        acc_history['test'].append(test_acc)
        loss_history['train'].append(train_loss)
        loss_history['test'].append(test_loss)

    # custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots')
    print(acc_history)
    print(loss_history)

In [ ]:
demo()

100%|██████████| 25/25 [05:47<00:00, 13.89s/it]

{'train': [tensor(19.46, device='cuda:0'), tensor(27.77, device='cuda:0'), tensor(36.31, device='cuda:0'), tensor(44.98, device='cuda:0'), tensor(50.22, device='cuda:0'), tensor(54.23, device='cuda:0'), tensor(59.08, device='cuda:0'), tensor(64.84, device='cuda:0'), tensor(70.45, device='cuda:0'), tensor(74.50, device='cuda:0'), tensor(76.86, device='cuda:0'), tensor(78.60, device='cuda:0'), tensor(79.91, device='cuda:0'), tensor(81.10, device='cuda:0'), tensor(82.07, device='cuda:0'), tensor(83.12, device='cuda:0'), tensor(83.90, device='cuda:0'), tensor(84.87, device='cuda:0'), tensor(85.55, device='cuda:0'), tensor(86.34, device='cuda:0'), tensor(87.03, device='cuda:0'), tensor(87.62, device='cuda:0'), tensor(88.11, device='cuda:0'), tensor(88.67, device='cuda:0'), tensor(89.11, device='cuda:0')], 'test': [tensor(24.67, device='cuda:0'), tensor(30.42, device='cuda:0'), tensor(40.53, device='cuda:0'), tensor(47.17, device='cuda:0'), tensor(51.62, device='cuda:0'), tensor(56.35, devic

# part 3









In [20]:
class demo_model2(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # kernel
        self.conv1 = torch.nn.Conv2d(3, 6, 5, stride=1, padding=0)
        self.conv2 = torch.nn.Conv2d(6, 16, 3, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(16, 32, 3, stride=1, padding=0)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=3200 + 256, out_features=1024)
        self.fc2 = nn.Linear(in_features=1024, out_features=128)
        self.fc3 = nn.Linear(in_features=128, out_features=10)

    # Forward Pass
    def forward(self, inputs, inputs2, debug=False):
        x = F.max_pool2d(F.relu(self.conv1(inputs)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 1)
        x = F.max_pool2d(F.relu(self.conv3(x)), 1)
        x0 = self.flatten(x)

        use_gpu = True
        device = torch.device("cuda" if torch.cuda.is_available() and use_gpu else "cpu")
        merged_input = torch.zeros(128, 3456)
        merged_input = merged_input.to(device)

        for i in range(x0.shape[0]):
            merged_input[i] = torch.cat((x0[i], inputs2[i]))


        x1 = self.fc1(merged_input)
        x1 = F.relu(x1)
        x2 = self.fc2(x1)
        penultimate = F.relu(x2)
        outputs = self.fc3(penultimate)

        if debug:
            print('inputs shape: ', inputs.shape) # inputs in shape [N, C, H, W]
            print('after flattening: ', x0.shape)
            print('Activations after 1st fully connected layer: ', x1.shape)
            print('Activations after 2nd fully connected layer: ', x2.shape)
            print('Output shape: ', outputs.shape)

        return outputs, penultimate

model = demo_model2()
print(model)

demo_model2(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3456, out_features=1024, bias=True)
  (fc2): Linear(in_features=1024, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=10, bias=True)
)


In [24]:
def train_one_epoch2(model: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn, triplet_loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.train() #
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
        if digit_labels.shape[0] != 128:
            continue
        images = images.to(device)
        features = features.to(device)
        digit_labels = digit_labels.to(device)
        domain_labels = domain_labels.to(device)

        outputs, penultimate = model(images, features) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, digit_labels) # Compute Loss

        loss2 = triplet_loss_fn(penultimate, domain_labels)
        landa = 0.1
        loss_total = loss + landa * loss2

        loss_total.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        running_corrects += torch.sum(preds == digit_labels)
        running_loss += loss_total.item()

    epoch_acc = (running_corrects / num_samples) * 100
    epoch_loss = (running_loss / num_batches)


    return epoch_acc, epoch_loss

## Evaluating Model:

In [25]:
def test_model2(model: nn.Module,
         dataloader: DataLoader, loss_fn, triplet_loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.eval() # you must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference.
    with torch.no_grad():
        for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
            if digit_labels.shape[0] != 128:
                continue
            images = images.to(device)
            features = features.to(device)
            digit_labels = digit_labels.to(device)
            domain_labels = domain_labels.to(device)

            outputs, penultimate = model(images, features) # Forward Pass
            loss = loss_fn(outputs, digit_labels) # Compute Loss
            loss2 = triplet_loss_fn(penultimate, domain_labels)
            print(loss2)
            landa = 0.1
            loss_total = loss + landa * loss2

            _, preds = torch.max(outputs, 1) #
            running_corrects += torch.sum(preds == digit_labels)
            running_loss += loss_total.item()

    test_acc = (running_corrects / num_samples) * 100
    test_loss = (running_loss / num_batches)

    return test_acc, test_loss

In [11]:
class triplet_loss(nn.Module):
    def __init__(self, margin: float = 0.005, device = torch.device("cuda")) -> None:
        super().__init__()
        """
        triplet loss implementation, explained in section 3 of project documentation.

        Args:
            margin: margin parameter in triplet loss (equation 5 in documentation)
                default value is 0.005, you may need to set different value for margin depending on your model.
            device: pass torch.device("cuda:0") to this parameter if you are using gpu.
        """
        self.margin = torch.tensor(margin)
        self.device = device

    # Compelete this function
    def forward(self, embeddings, labels):
        """
        embeddings: [M, d] M is the batch size, d is the dimension of the embeddings (dimension of the layer before the last layer)
        embeddings are supposed to be outputs of the layer before the last layer.

        labels: [M, ]
        """

        dp, dn = self.batch_hard_triplet_loss(embeddings, labels)
        sigma = 0
        for i in range(dp.shape[0]):
            result = max(dp[i]-dn[i]+self.margin, 0)
            sigma += result

        triplet_loss = sigma / labels.shape[0]

        return triplet_loss

    def batch_hard_triplet_loss(self, embeddings, labels):
        """
        Args:
            embeddings -> [N, d]
            labels -> [N, 1]

        returns:
            dp -> [N, 1] distance of furthest positive pair for each sample
            dn -> [N, 1] distance of closest negative pair for each sample
        """

        dists = self.euclidean_dist(embeddings, embeddings)
        # dists -> [N, N], square mat of all distances,
        # dists[i, j] is distance between sample[i] and sample[j]

        same_identity_mask = torch.eq(labels[:, None], labels[None, :])
        # [N, N], same_mask[i, j] = True when sample i and j have the same label

        negative_mask = torch.logical_not(same_identity_mask)
        # [N, N], negative_mask[i, j] = True when sample i and j have different label

        positive_mask = torch.logical_xor(same_identity_mask, torch.eye(labels.shape[0], dtype=torch.bool).to(self.device))
        # [N, N], same as same_identity mask, except diagonal is zero

        dp, _ = torch.max(dists * (positive_mask.int()), dim=1)

        dn = torch.zeros_like(dp)
        for i in range(dists.shape[0]):
            dn[i] = torch.min(dists[i, :][negative_mask[i, :]])

        return dp, dn

    def all_diffs(self, a, b):
        # a, b -> [N, d]
        # return -> [N, N, d]
        return a[:, None] - b[None, :]

    def euclidean_dist(self, embed1, embed2):
        # embed1, embed2 -> [N, d]
        # return [N, N] -> # get a square matrix of all diffs, diagonal is zero
        diffs = self.all_diffs(embed1, embed2)
        t1 = torch.square(diffs)
        t2 = torch.sum(t1, dim=-1)
        return torch.sqrt(t2 + 1e-12)

In [27]:
def demo2():
    batch_size = 128
    num_epochs = 20
    learning_rate = 0.005

    model = demo_model2()
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()
    triplet_loss_fn = triplet_loss()

    acc_history = {'train': [], 'test': []}
    loss_history = {'train': [], 'test': []}

    for epoch in trange(num_epochs):
        train_acc, train_loss = train_one_epoch2(model=model, optim=optimizer, dataloader=full_dataloaders['train'], loss_fn=cross_entropy, triplet_loss_fn=triplet_loss_fn)
        test_acc, test_loss = test_model2(model=model, dataloader=full_dataloaders['test'], loss_fn=cross_entropy, triplet_loss_fn=triplet_loss_fn)

        acc_history['train'].append(train_acc)
        acc_history['test'].append(test_acc)
        loss_history['train'].append(train_loss)
        loss_history['test'].append(test_loss)

    # custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots')
    print(acc_history)
    print(loss_history)

In [28]:
demo2()

100%|██████████| 20/20 [19:40<00:00, 59.01s/it]

{'train': [tensor(18.70, device='cuda:0'), tensor(26.69, device='cuda:0'), tensor(32.22, device='cuda:0'), tensor(36.97, device='cuda:0'), tensor(41.67, device='cuda:0'), tensor(46.64, device='cuda:0'), tensor(51.13, device='cuda:0'), tensor(54.53, device='cuda:0'), tensor(57.65, device='cuda:0'), tensor(60.86, device='cuda:0'), tensor(64.57, device='cuda:0'), tensor(68.79, device='cuda:0'), tensor(73.55, device='cuda:0'), tensor(77.37, device='cuda:0'), tensor(79.95, device='cuda:0'), tensor(82.45, device='cuda:0'), tensor(83.99, device='cuda:0'), tensor(85.28, device='cuda:0'), tensor(86.66, device='cuda:0'), tensor(87.58, device='cuda:0')], 'test': [tensor(23.70, device='cuda:0'), tensor(29.17, device='cuda:0'), tensor(34.06, device='cuda:0'), tensor(37.90, device='cuda:0'), tensor(42.79, device='cuda:0'), tensor(49.08, device='cuda:0'), tensor(52.92, device='cuda:0'), tensor(55.17, device='cuda:0'), tensor(60.36, device='cuda:0'), tensor(62.84, device='cuda:0'), tensor(66.94, devic

# Bonus score

In [8]:
class feature_producer_model(nn.Module):
    def __init__(self) -> None:
        super().__init__()

        # kernel
        self.conv1 = torch.nn.Conv2d(3, 6, 5, stride=1, padding=0)
        self.conv2 = torch.nn.Conv2d(6, 16, 3, stride=1, padding=0)
        self.conv3 = torch.nn.Conv2d(16, 32, 3, stride=1, padding=0)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=3200, out_features=256)

    # Forward Pass
    def forward(self, inputs, debug=False):
        x = F.max_pool2d(F.relu(self.conv1(inputs)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 1)
        x = F.max_pool2d(F.relu(self.conv3(x)), 1)
        x0 = self.flatten(x)
        outputs = self.fc1(x0)

        if debug:
            print('inputs shape: ', inputs.shape) # inputs in shape [N, C, H, W]
            print('after flattening: ', x0.shape)
            print('Activations after 1st fully connected layer: ', x1.shape)
            print('Activations after 2nd fully connected layer: ', x2.shape)
            print('Output shape: ', outputs.shape)

        return outputs

model = feature_producer_model()
print(model)

feature_producer_model(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=3200, out_features=256, bias=True)
)


In [9]:
from sklearn.model_selection import train_test_split

def train_one_epoch3(feature_producer_model: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    feature_producer_model.train() #
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
        if digit_labels.shape[0] != 128:
            continue
        images = images.to(device)
        features = features.to(device)
        X_train, X_test, y_train, y_test = train_test_split(images, features, test_size=0.20, random_state=42)

        outputs = feature_producer_model(X_train) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, y_train) # Compute Loss

        loss.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        running_corrects += 1 - torch.linalg.norm(outputs-y_train) / 256
        running_loss += loss.item()

    epoch_acc = (running_corrects / num_samples) * 100
    epoch_loss = (running_loss / num_batches)


    return epoch_acc, epoch_loss

# Evaluating Model

In [10]:
def test_model3(feature_producer_model: nn.Module,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    feature_producer_model.eval() # you must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference.
    with torch.no_grad():
        for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
            if digit_labels.shape[0] != 128:
                continue
            images = images.to(device)
            features = features.to(device)

            X_train, X_test, y_train, y_test = train_test_split(images, features, test_size=0.20, random_state=42)
            outputs = feature_producer_model(X_test) # Forward Pass
            loss = loss_fn(outputs, y_test) # Compute Loss

            running_corrects += 1 - torch.linalg.norm(outputs-y_test) / 256
            running_loss += loss.item()

    test_acc = (running_corrects / num_samples) * 100
    test_loss = (running_loss / num_batches)

    return test_acc, test_loss

In [17]:
def demo3():
    batch_size = 128
    num_epochs = 10
    learning_rate = 0.005

    model = feature_producer_model()
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()

    acc_history = {'train': [], 'test': []}
    loss_history = {'train': [], 'test': []}

    for epoch in trange(num_epochs):
        train_acc, train_loss = train_one_epoch3(feature_producer_model=model, optim=optimizer, dataloader=full_dataloaders['train'], loss_fn=cross_entropy)
        test_acc, test_loss = test_model3(feature_producer_model=model, dataloader=full_dataloaders['test'], loss_fn=cross_entropy)

        acc_history['train'].append(train_acc)
        acc_history['test'].append(test_acc)
        loss_history['train'].append(train_loss)
        loss_history['test'].append(test_loss)
    torch.save(model, '/content/model_scripted.pt')

    # custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots')
    # print(acc_history)
    # print(loss_history)

In [18]:
demo3()

100%|██████████| 10/10 [00:55<00:00,  5.53s/it]

{'train': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan], 'test': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}


In [19]:
def train_one_epoch4(model: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.train() #
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
        if digit_labels.shape[0] != 128:
            continue
        images = images.to(device)
        features = features.to(device)
        digit_labels = digit_labels.to(device)

        outputs = model(images, features) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, digit_labels) # Compute Loss

        loss.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        running_corrects += torch.sum(preds == digit_labels)
        running_loss += loss.item()

    epoch_acc = (running_corrects / num_samples) * 100
    epoch_loss = (running_loss / num_batches)


    return epoch_acc, epoch_loss

In [20]:
def test_model4(model: nn.Module, model_features: nn.Module,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.eval() # you must call `model.eval()` to set dropout and batch normalization layers to evaluation mode before running inference.
    with torch.no_grad():
        for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
            if digit_labels.shape[0] != 128:
                continue
            images = images.to(device)
            features = features.to(device)
            features = model_features(images)
            digit_labels = digit_labels.to(device)

            outputs = model(images, features) # Forward Pass
            loss = loss_fn(outputs, digit_labels) # Compute Loss

            _, preds = torch.max(outputs, 1) #
            running_corrects += torch.sum(preds == digit_labels)
            running_loss += loss.item()

    test_acc = (running_corrects / num_samples) * 100
    test_loss = (running_loss / num_batches)

    return test_acc, test_loss

In [23]:
def demo4():
    batch_size = 128
    num_epochs = 10
    learning_rate = 0.005

    model = demo_model()
    model = model.to(device)
    my_model = torch.load('/content/model_scripted.pt')

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()

    acc_history = {'train': [], 'test': []}
    loss_history = {'train': [], 'test': []}

    for epoch in trange(num_epochs):
        train_acc, train_loss = train_one_epoch4(model=model, optim=optimizer, dataloader=full_dataloaders['train'], loss_fn=cross_entropy)
        test_acc, test_loss = test_model4(model=model, model_features=my_model, dataloader=full_dataloaders['test_missing'], loss_fn=cross_entropy)

        acc_history['train'].append(train_acc)
        acc_history['test'].append(test_acc)
        loss_history['train'].append(train_loss)
        loss_history['test'].append(test_loss)

    # custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots')
    print(acc_history)
    # print(loss_history)

In [24]:
demo4()

100%|██████████| 10/10 [02:22<00:00, 14.27s/it]

{'train': [tensor(21.10, device='cuda:0'), tensor(27.78, device='cuda:0'), tensor(37.38, device='cuda:0'), tensor(45.21, device='cuda:0'), tensor(49.49, device='cuda:0'), tensor(52.48, device='cuda:0'), tensor(55.87, device='cuda:0'), tensor(59.71, device='cuda:0'), tensor(63.88, device='cuda:0'), tensor(68.40, device='cuda:0')], 'test': [tensor(10.56, device='cuda:0'), tensor(10.54, device='cuda:0'), tensor(10.56, device='cuda:0'), tensor(10.52, device='cuda:0'), tensor(10.56, device='cuda:0'), tensor(10.55, device='cuda:0'), tensor(10.55, device='cuda:0'), tensor(10.56, device='cuda:0'), tensor(10.56, device='cuda:0'), tensor(10.54, device='cuda:0')]}
{'train': [2.258704131600191, 2.137253085433293, 1.9428035961285328, 1.7148860970031479, 1.5386524350404231, 1.425506397859374, 1.3250914667206786, 1.2251551150004747, 1.1193377108716254, 0.9908559388443351], 'test': [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan]}


In [42]:
def train_one_epoch5(model: nn.Module, optim: torch.optim.Optimizer,
         dataloader: DataLoader, loss_fn):

    # utils
    num_samples = len(dataloader.dataset)
    num_batches = len(dataloader)
    running_corrects = 0
    running_loss = 0.0

    model.train() #
    for batch_indx, (images, features, domain_labels, digit_labels) in enumerate(dataloader): # Get a batch of Data
        if digit_labels.shape[0] != 128:
            continue
        images = images.to(device)
        features = features.to(device)
        digit_labels = digit_labels.to(device)

        outputs = model(images, features) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, digit_labels) # Compute Loss

        loss.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        running_corrects += torch.sum(preds == digit_labels)
        running_loss += loss.item()

        features = torch.zeros_like(features).to(device)
        outputs = model(images, features) # Forward Pass, [N, 10]
        loss = loss_fn(outputs, digit_labels) # Compute Loss

        loss.backward() # Compute Gradients
        optim.step() # Update parameters
        optim.zero_grad() # zero the parameter's gradients

        _, preds = torch.max(outputs, dim=1) # Explain, [N]
        running_corrects += torch.sum(preds == digit_labels)
        running_loss += loss.item()

    epoch_acc = (running_corrects / num_samples) * 100 / 2
    epoch_loss = (running_loss / num_batches) / 2


    return epoch_acc, epoch_loss

In [44]:
def demo5():
    batch_size = 128
    num_epochs = 10
    learning_rate = 0.005

    model = demo_model()
    model = model.to(device)

    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
    cross_entropy = nn.CrossEntropyLoss()

    acc_history = {'train': [], 'test': [], 'test_missing': []}
    loss_history = {'train': [], 'test': [], 'test_missing': []}

    for epoch in trange(num_epochs):
        train_acc, train_loss = train_one_epoch5(model=model, optim=optimizer, dataloader=full_dataloaders['train'], loss_fn=cross_entropy)
        test_acc, test_loss = test_model(model=model, dataloader=full_dataloaders['test'], loss_fn=cross_entropy)
        test_acc_missing, test_loss_missing = test_model(model=model, dataloader=full_dataloaders['test_missing'], loss_fn=cross_entropy)

        acc_history['train'].append(train_acc)
        acc_history['test'].append(test_acc)
        acc_history['test_missing'].append(test_acc_missing)
        loss_history['train'].append(train_loss)
        loss_history['test'].append(test_loss)
        loss_history['test_missing'].append(test_loss_missing)

    # custom_plot_training_stats(acc_history, loss_history, ['train', 'test'], title='demp', dir='demo_plots')
    print(acc_history)
    print(loss_history)

In [45]:
demo5()

100%|██████████| 10/10 [04:10<00:00, 25.10s/it]

{'train': [tensor(17.92, device='cuda:0'), tensor(27.39, device='cuda:0'), tensor(55.79, device='cuda:0'), tensor(67.74, device='cuda:0'), tensor(74.40, device='cuda:0'), tensor(78.33, device='cuda:0'), tensor(80.87, device='cuda:0'), tensor(82.78, device='cuda:0'), tensor(84.47, device='cuda:0'), tensor(85.87, device='cuda:0')], 'test': [tensor(24.35, device='cuda:0'), tensor(47.45, device='cuda:0'), tensor(62.35, device='cuda:0'), tensor(68.13, device='cuda:0'), tensor(74.62, device='cuda:0'), tensor(76.68, device='cuda:0'), tensor(78.32, device='cuda:0'), tensor(79.39, device='cuda:0'), tensor(80.99, device='cuda:0'), tensor(81.25, device='cuda:0')], 'test_missing': [tensor(13.12, device='cuda:0'), tensor(45.37, device='cuda:0'), tensor(62.05, device='cuda:0'), tensor(67.83, device='cuda:0'), tensor(74.40, device='cuda:0'), tensor(76.56, device='cuda:0'), tensor(78.35, device='cuda:0'), tensor(79.42, device='cuda:0'), tensor(81.04, device='cuda:0'), tensor(81.16, device='cuda:0')]}
